In [1]:
#引入一個開源運算框架 
! git clone https://github.com/pjreddie/darknet.git
# 修改make file
! head darknet/Makefile
!sed -i "s/GPU=0/GPU=1/g" darknet/Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" darknet/Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" darknet/Makefile
#編譯
! cd darknet; make

Cloning into 'darknet'...
remote: Enumerating objects: 5913, done.
remote: Total 5913 (delta 0), reused 0 (delta 0), pack-reused 5913
Receiving objects: 100% (5913/5913), 6.34 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (3922/3922), done.
GPU=0
CUDNN=0
OPENCV=0
OPENMP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52]
mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/incl

In [2]:
#抓取訓練資料
!wget https://tinyurl.com/y6d6p24o

--2020-08-20 09:09:45--  https://tinyurl.com/y6d6p24o
Resolving tinyurl.com (tinyurl.com)... 104.20.139.65, 104.20.138.65, 172.67.1.225, ...
Connecting to tinyurl.com (tinyurl.com)|104.20.139.65|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www2.cs.ccu.edu.tw/~chh90u/medical-masks-dataset.zip [following]
--2020-08-20 09:10:02--  http://www2.cs.ccu.edu.tw/~chh90u/medical-masks-dataset.zip
Resolving www2.cs.ccu.edu.tw (www2.cs.ccu.edu.tw)... 140.123.101.222
Connecting to www2.cs.ccu.edu.tw (www2.cs.ccu.edu.tw)|140.123.101.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150814019 (144M) [application/zip]
Saving to: ‘y6d6p24o’

y6d6p24o            100%[===================>] 143.83M  11.3MB/s    in 14s     

2020-08-20 09:10:16 (10.6 MB/s) - ‘y6d6p24o’ saved [150814019/150814019]



In [4]:
#更新成可用的檔名
!mv /content/y6d6p24o /content/medical-masksdataset.zip

In [5]:
#解壓縮
! unzip /content/medical-masksdataset.zip > data_unzip.log

In [6]:
#查看圖片對應的XML 用支援YOLO的標註器完成
! cat /content/labels/012106.jpg_1140x855.xml

<annotation>
	<folder>images</folder>
	<filename>012106.jpg_1140x855.jpg</filename>
	<path>P:\mask_mouth\images\012106.jpg_1140x855.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1140</width>
		<height>855</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>good</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>654</xmin>
			<ymin>275</ymin>
			<xmax>910</xmax>
			<ymax>517</ymax>
		</bndbox>
	</object>
</annotation>


In [7]:
#建立資料夾
! rm -rf /content/yolo
! mkdir /content/yolo

In [8]:
#建置給YOLO format的函數-> 把data 的 label 給yolo用
from bs4 import BeautifulSoup
import os
import shutil

status_dic = {'good':2,'bad':1,'none':0}
def getYoloFormat(filename,label_path,img_path,yolo_path,newname):
  with open(label_path+ filename, 'r') as f:
    # read xml
    soup = BeautifulSoup(f.read(),'xml')
    imgname = soup.select_one('filename').text
    image_w = int(soup.select_one('width').text)
    image_h = int(soup.select_one('height').text)
    ary = []
    for obj in soup.select('object'):
      #轉換標籤位置
      xmin = int(obj.select_one('xmin').text)
      xmax = int(obj.select_one('xmax').text)
      ymin = int(obj.select_one('ymin').text)
      ymax = int(obj.select_one('ymax').text)
      objclass = status_dic.get(obj.select_one('name').text)
      #照YOLO的格式訂好x,y,w,h
      x = (xmin + (xmax-xmin)/2) * 1.0 / image_w
      y = (ymin + (ymax-ymin)/2) * 1.0 / image_h
      w= (xmax-xmin) * 1.0 / image_w
      h = (ymax-ymin) * 1.0 / image_h
      ary.append(' '.join([str(objclass), str(x), str(y), str(w), str(h)]))
    #圖片本來在image裡面，把圖片移到yolo資料夾下
    #同時把yolo參數檔寫到yolo之下
    if os.path.exists(img_path + imgname):
        shutil.copyfile(img_path + imgname, yolo_path + newname + '.jpg')
        with open(yolo_path + newname + '.txt', 'w') as f:
          f.write('\n'.join(ary))



In [9]:
#使用函數 把訓練圖片和標籤(處理後變txt)放入同一個目錄下
import os
labelpath = '/content/labels/'
imgpath = '/content/images/'
yolopath = '/content/yolo/'
ary = []
for idx,f in enumerate(os.listdir(labelpath)):
  try:
    getYoloFormat(f,labelpath,imgpath,yolopath,str(idx))
  except Exception as e:
    print(e)

float division by zero


In [10]:
#看一下標籤轉換後的樣子
! cat /content/yolo/0.txt

2 0.578 0.616 0.468 0.46

In [11]:
#建立資料夾
! rm -rf /content/cfg_mask
! mkdir /content/cfg_mask

In [12]:
#計算資料總數 且8:2
import os
datasets = ['/content/yolo/' + f for f in os.listdir('/content/yolo/') if not f.endswith('.txt')]
len(datasets)*0.8

541.6

In [13]:
#資料分兩邊擺
with open('/content/cfg_mask/train.txt','w') as f:
  f.write('\n'.join(datasets[0:540]))
with open('/content/cfg_mask/test.txt','w') as f:
  f.write('\n'.join(datasets[540:]))

In [14]:
#取得設定檔
!wget https://drive.google.com/u/1/uc?id=1x1Hz131ZZsQKw55vM8CeOp_L-hRVlcbn&export=download
!wget https://drive.google.com/u/1/uc?id=1yuK3_SuS5YWZpV7fVWyVK6c_Au43nhus&export=download

--2020-08-20 09:20:57--  https://drive.google.com/u/1/uc?id=1x1Hz131ZZsQKw55vM8CeOp_L-hRVlcbn
Resolving drive.google.com (drive.google.com)... 74.125.142.102, 74.125.142.139, 74.125.142.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-9c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n2tkprk3ce32kfq3nk81aq8gpc9fdm6i/1597915200000/07559751869126818971/*/1x1Hz131ZZsQKw55vM8CeOp_L-hRVlcbn [following]
--2020-08-20 09:20:58--  https://doc-0s-9c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n2tkprk3ce32kfq3nk81aq8gpc9fdm6i/1597915200000/07559751869126818971/*/1x1Hz131ZZsQKw55vM8CeOp_L-hRVlcbn
Resolving doc-0s-9c-docs.googleusercontent.com (doc-0s-9c-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-0s-9c-docs.googleusercontent.com (doc-0s-9c-docs.googleusercontent.com)|74

In [15]:
#用旁邊改名 再移動位置
! mv '/content/obj.data' '/content/cfg_mask/obj.data'
! mv '/content/obj.names' '/content/cfg_mask/obj.names'

In [16]:
#看一下設定檔
! head /content/cfg_mask/obj.names
#none
#bad
#good

none
bad
good


In [17]:
# 建立權重資料夾
! mkdir /content/cfg_mask/weights

In [18]:
#修改參數
#先複製原來的檔案到 cfg_mask
! cp /content/darknet/cfg/yolov3-tiny.cfg /content/cfg_mask/

In [21]:
#看一下設定檔
! sed -n -e 127p -e 135p -e 171p -e 177p /content/cfg_mask/yolov3-tiny.cfg

filters=24
classes=3
filters=24
classes=3


In [20]:
#我們的class 只有三類 戴 沒戴 戴的不正常
! sed -i '127s/255/24/' /content/cfg_mask/yolov3-tiny.cfg
! sed -i '135s/80/3/' /content/cfg_mask/yolov3-tiny.cfg
! sed -i '171s/255/24/' /content/cfg_mask/yolov3-tiny.cfg
! sed -i '177s/80/3/' /content/cfg_mask/yolov3-tiny.cfg

In [23]:
# 抓取模型 (我們的模型構築在已經訓練好的一個模型上) 再複製到資料夾 先跑上面 再跑下面
#!wget https://pjreddie.com/media/files/darknet53.conv.74
! cp "/content/darknet53.conv.74" /content/cfg_mask/

In [ ]:
#進行訓練
! darknet/darknet detector train /content/cfg_mask/obj.data /content/cfg_mask/yolov3-tiny.cfg /content/cfg_mask/darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Loaded: 0.000050 seconds
Region 16 Avg IOU: 0.785605, Class: 0.993015, Obj: 0.959999, No Obj: 0.002578, .5R: 1.000000, .75R: 1.000000,  count: 1
Region 23 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000260, .5R: -nan, .75R: -nan,  count: 0
100011: 0.087468, 1.854550 avg, 0.001000 rate, 0.006716 seconds, 100011 images
Loaded: 0.010798 seconds
Region 16 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000002, .5R: -nan, .75R: -nan,  count: 0
Region 23 Avg IOU: 0.550502, Class: 0.999200, Obj: 0.909647, No Obj: 0.003087, .5R: 0.600000, .75R: 0.200000,  count: 5
100012: 1.124686, 1.781564 avg, 0.001000 rate, 0.006411 seconds, 100012 images
Loaded: 0.052528 seconds
Region 16 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.001458, .5R: -nan, .75R: -nan,  count: 0
Region 23 Avg IOU: 0.728043, Class: 0.984317, Obj: 0.735671, No Obj: 0.008933, .5R: 1.000000, .75R: 0.384615,  count: 13
100013: 3.844884, 1.987896 avg, 0.001000 rate, 0.008456

In [31]:
#進行辨識
! cd darknet; ./darknet detector test /content/cfg_mask/obj.data /content/cfg_mask/yolov3-tiny.cfg /content/cfg_mask/weights/yolov3-tiny_900.weights /content/images/001_1024.jpeg

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con